# Template Notebook for testing an Island Model GA with Artists/Styles on an island
Notebook Version: 0.41 (26/03/2024)
* can now visualize best solution in grid by identifier

## Google Colab Setup

In [ ]:
# Google Colab: Execute this to install packages and setup drive
!pip install "evolutionary[all] @ git+https://git@github.com/malthee/evolutionary-diffusion.git"

In [ ]:
# Mount drive to save results
from google.colab import drive
import evolutionary_imaging.processing as ip
drive.mount("/content/drive")
base_path = "/content/drive/MyDrive/evolutionary/"
ip.RESULTS_FOLDER = base_path + ip.RESULTS_FOLDER

In [ ]:
# Check if GPU is available
import torch
print(torch.cuda.is_available())

## Project Setup

In [ ]:
from evolutionary.plotting import plot_fitness_statistics
import evolutionary_imaging.processing as ip
from diffusers.utils import logging
from evolutionary_imaging.processing import create_animation_from_generations, create_generation_image_grid, save_images_from_generation
import torch
import os

In [ ]:
logging.disable_progress_bar() # Or else your output will be full of progress bars
logging.set_verbosity_error() # Enable again if you are having problems
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To remove warning of libraries using tokenizers
# Change the results folder for images if you want to 
# ip.RESULTS_FOLDER = 'choose_your_destination'

class SaveImagesPostEvaluation:  # Class to save images and difference between islands; used to allow pickling
    def __init__(self, ident):
        self.ident = ident

    def __call__(self, g, a):
        return save_images_from_generation(a.population, g, self.ident)

# Check torch random state, used across all libraries. Caution setting fixed seeds as it affects not only generation but also variation.
print(torch.random.get_rng_state())

In [ ]:
from evolutionary_prompt_embedding.argument_types import PooledPromptEmbedData
from evolutionary_prompt_embedding.image_creation import SDXLPromptEmbeddingImageCreator
from evolutionary_prompt_embedding.variation import \
    UniformGaussianMutatorArguments, PooledUniformGaussianMutator, PooledArithmeticCrossover, PooledUniformCrossover
from evolutionary_prompt_embedding.value_ranges import SDXLTurboEmbeddingRange, SDXLTurboPooledEmbeddingRange
from evolutionary.evolutionary_selectors import TournamentSelector, RouletteWheelSelector, RankSelector
from evolutionary.evolution_base import CappedEvaluator, GoalDiminishingEvaluator
from evolutionary.algorithms.island_model import IslandModel
from evolutionary.algorithms.ga import GeneticAlgorithm
from evolutionary_imaging.evaluators import AIDetectionImageEvaluator, AestheticsImageEvaluator

population_size = 20
num_generations = 5
batch_size = 1
elitism = None
inference_steps = 4

epochs = [
    "Prehistoric to Ancient Civilizations Art",
    "Classical Antiquity to Early Medieval Art",
    "Renaissance to Mannerism",
    "Baroque to Rococo",
    "Neoclassicism to Romanticism",
    "Realism to Post-Impressionism",
    "Modern Art",
    "Contemporary and Postmodern Art"
]

visual_arts_by_epoch = [
    [
        "Lascaux Cave Paintings",
        "Venus of Willendorf",
        "Terracotta Army",
        "Great Sphinx of Giza",
        "Moai Statues of Easter Island",
        "Aboriginal Rock Art",
        "Ajanta Caves Frescoes",
        "Sumerian Statuettes from the Temple of Abu",
        "Ancient Egyptian Book of the Dead",
        "Cycladic Figurines",
        "Buddhist Stupas of Sanchi",
        "Cave Paintings of Bhimbetka"
    ],
    [
        "Parthenon Friezes",
        "Frescoes of Pompeii",
        "Buddhist Statues of Bamiyan",
        "Book of Kells",
        "Hagia Sophia Mosaics",
        "Sutton Hoo Ship Burial",
        "Byzantine Icons of Christ Pantocrator",
        "Toreutics of the Scythians",
        "The Colosseum",
        "Mask of Tutankhamun",
        "Olmec Colossal Heads",
        "Terracotta Warriors"
    ],
    [
        "Mona Lisa by Leonardo da Vinci",
        "The School of Athens by Raphael",
        "The Sistine Chapel Ceiling by Michelangelo",
        "The Arnolfini Portrait by Jan van Eyck",
        "Alhambra's Islamic Calligraphy and Tile Work",
        "Benin Bronzes",
        "The Birth of Venus by Botticelli",
        "Kabuki Theater Woodblock Prints",
        "David by Michelangelo",
        "Ghent Altarpiece by Jan van Eyck",
        "The Garden of Earthly Delights by Hieronymus Bosch",
        "The Tempest by Giorgione"
    ],
    [
        "The Night Watch by Rembrandt",
        "The Ecstasy of Saint Teresa by Bernini",
        "Las Meninas by Velázquez",
        "The Swing by Fragonard",
        "Girl with a Pearl Earring by Vermeer",
        "Works of Artemisia Gentileschi",
        "Ukiyo-e Prints by Hokusai",
        "Palace of Versailles",
        "St. Paul's Cathedral",
        "The Embarkation for Cythera by Watteau",
        "The Death of Sardanapalus by Delacroix",
        "Self-Portrait with Thorn Necklace and Hummingbird by Frida Kahlo"
    ],
    [
        "Liberty Leading the People by Delacroix",
        "The Third of May 1808 by Goya",
        "The Death of Socrates by David",
        "The Raft of the Medusa by Géricault",
        "The Hay Wain by Constable",
        "Wanderer above the Sea of Fog by Friedrich",
        "The Tiger Hunt by Rubens",
        "The Great Wave off Kanagawa by Hokusai",
        "Saturn Devouring His Son by Goya",
        "Oath of the Horatii by David",
        "The Stone Breakers by Courbet",
        "American Gothic by Grant Wood"
    ],
    [
        "Olympia by Manet",
        "The Stone Breakers by Courbet",
        "Starry Night by Van Gogh",
        "The Scream by Munch",
        "A Sunday on La Grande Jatte by Seurat",
        "Tahitian Women on the Beach by Gauguin",
        "The Burghers of Calais by Rodin",
        "The Peasant Wedding by Bruegel the Elder",
        "The Balcony by Manet",
        "The Kiss by Gustav Klimt",
        "The Sleeping Gypsy by Henri Rousseau",
        "The Ladies of Avignon by Picasso"
    ],
    [
        "Les Demoiselles d'Avignon by Picasso",
        "The Persistence of Memory by Dalí",
        "Composition VIII by Kandinsky",
        "The Fountain by Duchamp",
        "American Gothic by Grant Wood",
        "The Migration Series by Jacob Lawrence",
        "Guernica by Picasso",
        "The Two Fridas by Kahlo",
        "Nighthawks by Edward Hopper",
        "Broadway Boogie Woogie by Mondrian",
        "Autumn Rhythm (Number 30) by Jackson Pollock",
        "Suprematist Composition by Kazimir Malevich",
        "The Red Studio by Henri Matisse",
        "Unique Forms of Continuity in Space by Umberto Boccioni",
        "The Blue Rider by Wassily Kandinsky"
    ],
    [
        "Campbell's Soup Cans by Andy Warhol",
        "The Physical Impossibility of Death in the Mind of Someone Living by Damien Hirst",
        "Untitled Film Stills by Cindy Sherman",
        "The Weather Project by Olafur Eliasson",
        "Vietnam Veterans Memorial by Maya Lin",
        "Spiral Jetty by Robert Smithson",
        "A Subtlety by Kara Walker",
        "The Flower Thrower by Banksy",
        "Balloon Dog by Jeff Koons",
        "Shibboleth by Doris Salcedo",
        "Sunflower Seeds by Ai Weiwei",
        "The Shark by Damien Hirst",
        "Rain Room by Random International",
        "The Basket of Apples by Paul Cézanne"
    ]
]

# Initialize GA instances
embedding_range = SDXLTurboEmbeddingRange()
pooled_embedding_range = SDXLTurboPooledEmbeddingRange()
# Create the necessary components for the genetic algorithm
creator = SDXLPromptEmbeddingImageCreator(batch_size=batch_size, inference_steps=inference_steps)
# Above this score the AestheticsImageEvaluator is biased to specific styles, use it more as quality control and leave it open 
evaluator = GoalDiminishingEvaluator(AestheticsImageEvaluator(), 5.5, 2)
crossover = PooledArithmeticCrossover(crossover_rate=0.75, crossover_rate_pooled=0.75) # Keep original styles more
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.05, mutation_strength=2.5, 
                                                     clamp_range=(embedding_range.minimum, embedding_range.maximum)) 
mutation_arguments_pooled = UniformGaussianMutatorArguments(mutation_rate=0.05, mutation_strength=0.4, 
                                                            clamp_range=(pooled_embedding_range.minimum, pooled_embedding_range.maximum))
mutator = PooledUniformGaussianMutator(mutation_arguments, mutation_arguments_pooled)
selector = TournamentSelector(tournament_size=3)

ga_instances = []

for i, epoch in enumerate(visual_arts_by_epoch):
    work_count = len(epoch)
    init_args = [creator.arguments_from_prompt(epoch[i % work_count]) for i in range(population_size)] 
    save_images_post_evaluation = SaveImagesPostEvaluation(i)
 
    # Create and run the genetic algorithm
    ga_instances.append(GeneticAlgorithm(
        population_size=population_size,
        num_generations=num_generations,
        solution_creator=creator,
        evaluator=evaluator,
        mutator=mutator,
        crossover=crossover,
        selector=selector,
        initial_arguments=init_args,
        elitism_count=elitism,
        post_evaluation_callback=save_images_post_evaluation,
    ))

In [ ]:
island_model = IslandModel(
    ga_instances,
    migration_size=2,
    migration_interval=5,
)

In [ ]:
best_solutions = island_model.run()

In [ ]:
from diffusers.utils import make_image_grid

# Show best solution
for i, best_solution in enumerate(best_solutions):
    print(f"Best solution for epoch {epochs[i]}: {best_solution.fitness}")

make_image_grid([image for solution in best_solutions for image in solution.result.images], 2, batch_size * len(best_solutions) // 2)

## Visualize the evolution

In [ ]:
for gen in range(num_generations):
    create_generation_image_grid(gen, max_images=10, label_fontsize=8, ident_mapper=epochs, group_by_ident=True)

In [ ]:
video_loc = create_animation_from_generations(num_generations)
print(video_loc)

## Plot fitness statistics

In [ ]:
plot_fitness_statistics(num_generations, island_model.best_fitness, island_model.worst_fitness, island_model.avg_fitness)

In [ ]:
!jupyter nbconvert --to html ga_notebook.ipynb

### Save the run to disk

In [ ]:
import pickle
import os
from datetime import datetime

os.makedirs("saved_runs", exist_ok=True)
output_file = os.path.join("saved_runs", f"island_model_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")
with open(output_file, "wb") as f:
    pickle.dump(island_model, f)
print(f"Run saved to {output_file}")
    

### Load the run from disk 
Notebook and library versions should match with the saved run

In [ ]:
import pickle
import os

with open(os.path.join("saved_runs", "insert_filename.pkl"), "rb") as f:
    island_model = pickle.load(f)

## Fallback functions for when something went wrong

### Access Best Solution from Disk

In [ ]:
import os
import glob
import evolutionary_imaging.processing as ip
from PIL import Image

num_generations = 22  # Set this to the number of generations you ran (if you didn't finish)
generation_dir = os.path.join(ip.RESULTS_FOLDER, f"{num_generations}")
image_files = glob.glob(os.path.join(generation_dir, "*.png"))
image_files.sort(key=ip.fitness_filename_sorting_key, reverse=True)
print(image_files[0])
Image.open(image_files[0])

### ffmpeg is not installed, create GIF instead

In [ ]:
from evolutionary_imaging.processing import create_animation_from_generations_pil
video_loc = create_animation_from_generations_pil(num_generations)
print(video_loc)